![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/accelerated-models/accelerated-models-object-detection.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Azure ML Hardware Accelerated Object Detection

This tutorial will show you how to deploy an object detection service based on the SSD-VGG model in just a few minutes using the Azure Machine Learning Accelerated AI service.

We will use the SSD-VGG model accelerated on an FPGA. Our Accelerated Models Service handles translating deep neural networks (DNN) into an FPGA program.

The steps in this notebook are: 
1. [Setup Environment](#set-up-environment)
* [Construct Model](#construct-model)
    * Image Preprocessing
    * Featurizer
    * Save Model
    * Save input and output tensor names
* [Create Image](#create-image)
* [Deploy Image](#deploy-image)
* [Test the Service](#test-service)
    * Create Client
    * Serve the model
* [Cleanup](#cleanup)

<a id="set-up-environment"></a>
## 1. Set up Environment
### 1.a. Imports

In [ ]:
import os
import tensorflow as tf

### 1.b. Retrieve Workspace
If you haven't created a Workspace, please follow [this notebook]("../../../configuration.ipynb") to do so. If you have, run the codeblock below to retrieve it. 

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

<a id="construct-model"></a>
## 2. Construct model
### 2.a. Image preprocessing
We'd like our service to accept JPEG images as input. However the input to SSD-VGG is a float tensor of shape \[1, 300, 300, 3\]. The first dimension is batch, then height, width, and channels (i.e. NHWC). To bridge this gap, we need code that decodes JPEG images and resizes them appropriately for input to SSD-VGG. The Accelerated AI service can execute TensorFlow graphs as part of the service and we'll use that ability to do the image preprocessing. This code defines a TensorFlow graph that preprocesses an array of JPEG images (as TensorFlow strings) and produces a tensor that is ready to be featurized by SSD-VGG.

**Note:** Expect to see TF deprecation warnings until we port our SDK over to use Tensorflow 2.0.

In [ ]:
# Input images as a two-dimensional tensor containing an arbitrary number of images represented a strings
import azureml.accel.models.utils as utils
tf.reset_default_graph()

in_images = tf.placeholder(tf.string)
image_tensors = utils.preprocess_array(in_images, output_width=300, output_height=300, preserve_aspect_ratio=False)
print(image_tensors.shape)

### 2.b. Featurizer
The SSD-VGG model is different from our other models in that it generates 12 tensor outputs. These corresponds to x,y displacements of the anchor boxes and the detection confidence (for 21 classes). Because these outputs are not convenient to work with, we will later use a pre-defined post-processing utility to transform the outputs into a simplified list of bounding boxes with their respective class and confidence.

For more information about the output tensors, take this example: the output tensor 'ssd_300_vgg/block4_box/Reshape_1:0' has a shape of [None, 37, 37, 4, 21]. This gives the pre-softmax confidence for 4 anchor boxes situated at each site of a 37 x 37 grid imposed on the image, one confidence score for each of the 21 classes. The first dimension is the batch dimension. Likewise, 'ssd_300_vgg/block4_box/Reshape:0' has shape [None, 37, 37, 4, 4] and encodes the (cx, cy) center shift and rescaling (sw, sh) relative to each anchor box. Refer to the [SSD-VGG paper](https://arxiv.org/abs/1512.02325) to understand how these are computed. The other 10 tensors are defined similarly.

In [ ]:
from azureml.accel.models import SsdVgg

saved_model_dir = os.path.join(os.path.expanduser('~'), 'models')
model_graph = SsdVgg(saved_model_dir, is_frozen = True)

print('SSD-VGG Input Tensors:')
for idx, input_name in enumerate(model_graph.input_tensor_list):
    print('{}, {}'.format(input_name, model_graph.get_input_dims(idx)))
    
print('SSD-VGG Output Tensors:')
for idx, output_name in enumerate(model_graph.output_tensor_list):
    print('{}, {}'.format(output_name, model_graph.get_output_dims(idx)))

ssd_outputs = model_graph.import_graph_def(image_tensors, is_training=False)

### 2.c. Save Model
Now that we loaded both parts of the tensorflow graph (preprocessor and SSD-VGG featurizer), we can save the graph and associated variables to a directory which we can register as an Azure ML Model.

In [ ]:
model_name = "ssdvgg"
model_save_path = os.path.join(saved_model_dir, model_name, "saved_model")
print("Saving model in {}".format(model_save_path))

output_map = {}
for i, output in enumerate(ssd_outputs):
    output_map['out_{}'.format(i)] = output

with tf.Session() as sess:
    model_graph.restore_weights(sess)
    tf.saved_model.simple_save(sess, 
                               model_save_path, 
                               inputs={'images': in_images}, 
                               outputs=output_map)

### 2.d. Important! Save names of input and output tensors

These input and output tensors that were created during the preprocessing and classifier steps are also going to be used when **converting the model** to an Accelerated Model that can run on FPGA's and for **making an inferencing request**. It is very important to save this information!

In [ ]:
input_tensors = in_images.name
# We will use the list of output tensors during inferencing
output_tensors = [output.name for output in ssd_outputs]
# However, for multiple output tensors, our AccelOnnxConverter will 
#    accept comma-delimited strings (lists will cause error)
output_tensors_str = ",".join(output_tensors)

print(input_tensors)
print(output_tensors)

<a id="create-image"></a>
## 3. Create AccelContainerImage
Below we will execute all the same steps as in the [Quickstart](./accelerated-models-quickstart.ipynb#create-image) to package the model we have saved locally into an accelerated Docker image saved in our workspace. To complete all the steps, it may take a few minutes. For more details on each step, check out the [Quickstart section on model registration](./accelerated-models-quickstart.ipynb#register-model).

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.image import Image
from azureml.accel import AccelOnnxConverter
from azureml.accel import AccelContainerImage

# Retrieve workspace
ws = Workspace.from_config()
print("Successfully retrieved workspace:", ws.name, ws.resource_group, ws.location, ws.subscription_id, '\n')

# Register model
registered_model = Model.register(workspace = ws,
                                  model_path = model_save_path,
                                  model_name = model_name)
print("Successfully registered: ", registered_model.name, registered_model.description, registered_model.version, '\n', sep = '\t')

# Convert model
convert_request = AccelOnnxConverter.convert_tf_model(ws, registered_model, input_tensors, output_tensors_str)
if convert_request.wait_for_completion(show_output = False):
    # If the above call succeeded, get the converted model
    converted_model = convert_request.result
    print("\nSuccessfully converted: ", converted_model.name, converted_model.url, converted_model.version, 
          converted_model.id, converted_model.created_time, '\n')
else:
    print("Model conversion failed. Showing output.")
    convert_request.wait_for_completion(show_output = True)

# Package into AccelContainerImage
image_config = AccelContainerImage.image_configuration()
# Image name must be lowercase
image_name = "{}-image".format(model_name)
image = Image.create(name = image_name,
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation()
print("Created AccelContainerImage: {} {} {}\n".format(image.name, image.creation_state, image.image_location))

<a id="deploy-image"></a>
## 4. Deploy image
Once you have an Azure ML Accelerated Image in your Workspace, you can deploy it to two destinations, to a Databox Edge machine or to an AKS cluster. 

### 4.a. Deploy to Databox Edge Machine using IoT Hub
See the sample [here](https://github.com/Azure-Samples/aml-real-time-ai/) for using the Azure IoT CLI extension for deploying your Docker image to your Databox Edge Machine.

### 4.b. Deploy to AKS Cluster
Same as in the [Quickstart section on image deployment](./accelerated-models-quickstart.ipynb#deploy-image), we are going to create an AKS cluster with FPGA-enabled machines, then deploy our service to it.
#### Create AKS ComputeTarget

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Uses the specific FPGA enabled VM (sku: Standard_PB6s)
# Standard_PB6s are available in: eastus, westus2, westeurope, southeastasia
prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_PB6s",
                                                    agent_count = 1, 
                                                    location = "eastus")

aks_name = 'aks-pb6-obj'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

Provisioning an AKS cluster might take awhile (15 or so minutes), and we want to wait until it's successfully provisioned before we can deploy a service to it. If you interrupt this cell, provisioning of the cluster will continue. You can re-run it or check the status in your Workspace under Compute.

In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

#### Deploy AccelContainerImage to AKS ComputeTarget

In [ ]:
%%time
from azureml.core.webservice import Webservice, AksWebservice

# Set the web service configuration (for creating a test service, we don't want autoscale enabled)
# Authentication is enabled by default, but for testing we specify False
aks_config = AksWebservice.deploy_configuration(autoscale_enabled=False,
                                                num_replicas=1,
                                                auth_enabled = False)

aks_service_name ='my-aks-service'

aks_service = Webservice.deploy_from_image(workspace = ws,
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)

<a id="test-service"></a>
## 5. Test the service
<a id="create-client"></a>
### 5.a. Create Client
The image supports gRPC and the TensorFlow Serving "predict" API. We will create a PredictionClient from the Webservice object that can call into the docker image to get predictions. If you do not have the Webservice object, you can also create [PredictionClient](https://docs.microsoft.com/en-us/python/api/azureml-accel-models/azureml.accel.predictionclient?view=azure-ml-py) directly.

**Note:** If you chose to use auth_enabled=True when creating your AksWebservice.deploy_configuration(), see documentation [here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice(class)?view=azure-ml-py#get-keys--) on how to retrieve your keys and use either key as an argument to PredictionClient(...,access_token=key).
**WARNING:** If you are running on Azure Notebooks free compute, you will not be able to make outgoing calls to your service. Try locating your client on a different machine to consume it.

In [ ]:
# Using the grpc client in AzureML Accelerated Models SDK
from azureml.accel import client_from_service

# Initialize AzureML Accelerated Models client
client = client_from_service(aks_service)

You can adapt the client [code](https://github.com/Azure/aml-real-time-ai/blob/master/pythonlib/amlrealtimeai/client.py) to meet your needs. There is also an example C# [client](https://github.com/Azure/aml-real-time-ai/blob/master/sample-clients/csharp).

The service provides an API that is compatible with TensorFlow Serving. There are instructions to download a sample client [here](https://www.tensorflow.org/serving/setup).

<a id="serve-model"></a>
### 5.b. Serve the model
The SSD-VGG model returns the confidence and bounding boxes for all possible anchor boxes. As mentioned earlier, we will use a post-processing routine to transform this into a list of bounding boxes (y1, x1, y2, x2) where x, y are fractional coordinates measured from left and top respectively. A respective list of classes and scores is also returned to tag each bounding box. Below we make use of this information to draw the bounding boxes on top the original image. Note that in the post-processing routine we select a confidence threshold of 0.5.

In [ ]:
import cv2
from matplotlib import pyplot as plt

colors_tableau = [(255, 255, 255), (31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),
                  (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),
                  (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),
                  (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
                  (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]


def draw_boxes_on_img(img, classes, scores, bboxes, thickness=2):
    shape = img.shape
    for i in range(bboxes.shape[0]):
        bbox = bboxes[i]
        color = colors_tableau[classes[i]]
        # Draw bounding box...
        p1 = (int(bbox[0] * shape[0]), int(bbox[1] * shape[1]))
        p2 = (int(bbox[2] * shape[0]), int(bbox[3] * shape[1]))
        cv2.rectangle(img, p1[::-1], p2[::-1], color, thickness)
        # Draw text...
        s = '%s/%.3f' % (classes[i], scores[i])
        p1 = (p1[0]-5, p1[1])
        cv2.putText(img, s, p1[::-1], cv2.FONT_HERSHEY_DUPLEX, 0.4, color, 1)

In [ ]:
import azureml.accel._external.ssdvgg_utils as ssdvgg_utils

result = client.score_file(path="meeting.jpg", input_name=input_tensors, outputs=output_tensors)
classes, scores, bboxes = ssdvgg_utils.postprocess(result, select_threshold=0.5)

img = cv2.imread('meeting.jpg', 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
draw_boxes_on_img(img, classes, scores, bboxes)
plt.imshow(img)

<a id="cleanup"></a>
## 6. Cleanup
It's important to clean up your resources, so that you won't incur unnecessary costs. In the [next notebook](./accelerated-models-training.ipynb) you will learn how to train a classfier on a new dataset using transfer learning.

In [ ]:
aks_service.delete()
aks_target.delete()
image.delete()
registered_model.delete()
converted_model.delete()